We'll update the database again. There seem to be some issues associated with importing vfeed.config that running the script resolves. Should make this mroe robust in the future...

In [1]:
%run -e vfeed/vfeedcli.py update

import sqlite3 
import vfeed.config as config
import pandas as pd

configData = config.database['primary'] 
vfeed_db  = configData['vfeed_db']
conn = sqlite3.connect(vfeed_db)

[info] checking for the latest vfeed.db 
[progress 100 %] receiving 49 out of 49 Bytes of update.dat 
[info] You have the latest vfeed.db vulnerability database
[info] Cleaning compressed database and update file


NameError: global name 'exit' is not defined

In [15]:
references = pd.read_sql('SELECT * FROM cve_reference', conn)
references.head()

,refsource,refname,cveid
0,CERT-VN,http://www.kb.cert.org/vuls/id/705004,CVE-2013-3611
1,XF,http://xforce.iss.net/xforce/xfdb/75789,CVE-2012-2951
2,BID,http://www.securityfocus.com/bid/53644,CVE-2012-2951
3,MISC,http://packetstormsecurity.org/files/112947/Pl...,CVE-2012-2951
4,DEBIAN,http://www.debian.org/security/2014/dsa-2895,CVE-2014-2750


In [6]:
import hashlib
import wget
import os.path
import time
import thread

In [5]:
def getfile(url):
    urlhash = hashlib.md5(url).hexdigest()
    outfile_name = 'refpages/page_%s.html'%urlhash

    try:
        if not os.path.isfile(outfile_name): 
            wget.download(url=url, out=outfile_name)
            #print 'got ', url
            time.sleep(5)
        else:
            #print 'had ', url
            pass
    except:
        pass

def getfileset(source, df):
    print 'starting %s'%source
    for index, row in df.iterrows():
        #print row
        getfile(row['refname'])
    print 'finished %s'%source


In [6]:
#if we wanted to get all of the sources, we could do it in parallel, 
# giving a new thread to each source. For now, we don't.
#for source, group in references.groupby(by='refsource'):
#    thread.start_new_thread( getfileset, (source, group,) )

In [ ]:
getfileset('MS', references[references['refsource']=='MS'])

In [28]:
import requests
url = references[references['refsource']=='MS']['refname'].iloc[0]
response = requests.get(url)


In [29]:
response.text

u'<!DOCTYPE html>\n\n<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml" lang="en">\n    <head><link rel="canonical" href="https://technet.microsoft.com/en-us/library/security/ms14-018.aspx" />\n        <title>Microsoft Security Bulletin MS14-018 - Critical</title>\n\n\n\n\n\n<meta name="DCS.dcsuri" content="/en-us/library/security/dn673562(d=default,l=en-us,v=security.10).aspx" />\n\n<meta name="NormalizedUrl" content="https://technet.microsoft.com/en-us/library/security/dn673562(d=default,l=en-us,v=security.10).aspx" />\n\n<meta name="ms.normalizedurl" content="https://technet.microsoft.com/en-us/library/security/dn673562(d=default,l=en-us,v=security.10).aspx" />\n\n<meta name="VotingContextUrl" content="https://technet.microsoft.com/en-us/library/security/dn673562(d=default,l=en-us,v=security.10).aspx" />\n\n<meta name="MN" content="FB361398-8:25:34 AM" />\n\n<meta name="Search.ShortId" content="dn673562" />\n\n<meta name="ms.shortidmsdn" content="dn673562" />\n\n<meta name="Ms.Loc

##Parse the downloaded references

In [3]:
import lxml.html as lh
import re

In [23]:
def load_cached(url):    
    urlhash = hashlib.md5(url).hexdigest()
    file_name = 'refpages/page_%s.html'%urlhash
    if not os.path.isfile(file_name): 
        return ''
    
    with open(file_name, 'r') as infile:
        text = infile.read()

    return text


In [22]:
relist = [re.compile('(?P<name>.+)\sof\s*(?P<org>.+)\sfor.+(CVE-\d{4}-\d{4})+'), # original finder # Nicolas Gregoire of Agarri for reporting the MSXML XSLT Vulnerability (CVE-2013-0007)
              re.compile('(?P<name>.+),\sworking\swith\s(?P<org>.+),\sfor.+(CVE-\d{4}-\d{4})+'), #tipping point means original discovery found in the wild.... #Vitaliy Toropov, working with Tipping Point'sZero Day Initiative, for reporting the S.DS.P Buffer Overflow Vulnerability (CVE-2013-0003)
              re.compile('(?P<name>.+)(?P<org>\s)for.+(CVE-\d{4}-\d{4})+'), #Kenichiro Katayama for reporting the Microsoft SSL Version 3 and TLS Protocol Security Feature Bypass Vulnerability (CVE-2013-0013)
              #re.compile('(.{1,70})\sof\s(.+)\sfor(\s)'), #Ollie Whitehouse of NCC Group for working with us on additional defense-in-depth changes included in this bulletin
              #re.compile('(.{1,70})(\s)for(\s)')# An anonymous researcher for working with us on defense-in-depth changes included in this bulletin
              ]

for url in references[references['refsource']=='MS']['refname'].iloc[:10]:
    print url
    
    content = load_cached(url)
    if content == '': continue
    #parse the page
    doc = lh.fromstring(content)
    text = doc.xpath("//*[text()[contains(.,'Acknowledgments')]]/../../ul/li")

    if text:
        for rex in relist:
            match = rex.match(text)
            if match: break

        if match:
            for cve in re.finditer('(CVE-\d{4}-\d{4})', text):
                print ('name', match.group(1), ' organization',  match.group(2),
                      'cve_id', cve.group(1), 'url', url)

    

http://technet.microsoft.com/security/bulletin/MS14-018
http://www.microsoft.com/technet/security/bulletin/ms98-008.asp
http://www.microsoft.com/technet/security/bulletin/ms98-002.mspx
http://www.microsoft.com/technet/security/bulletin/ms98-003.mspx
http://www.microsoft.com/technet/security/bulletin/ms98-009.mspx
http://www.microsoft.com/technet/security/bulletin/ms99-003.mspx
http://www.microsoft.com/technet/security/bulletin/ms99-002.asp
http://www.microsoft.com/technet/security/bulletin/ms99-004.mspx
http://www.microsoft.com/technet/security/bulletin/ms99-005.mspx
http://www.microsoft.com/technet/security/bulletin/ms99-006.mspx


In [25]:
lh.tostring(doc)

'<html><head>\n<title>Access Denied</title>\n</head><body>\n<h1>Access Denied</h1>\n \nYou don\'t have permission to access "http://www.microsoft.com/technet/security/bulletin/ms99-006.mspx" on this server.<p>\nReference #18.95f24317.1432734068.175715b\n</p></body>\n</html>'